<a href="https://colab.research.google.com/github/MatteoFusconi/Artificial-Intelligence-master/blob/main/Italian_market_ESN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install PyPortfolioOpt
!pip install pandas_ta
!pip install financedatabase
!pip install tensorflow_addons


In [3]:
from portfolio_opt import *
from utils import *
import numpy as np
import pandas as pd
from metrics import StockMeasures
import datetime as dt

# Italian market portfolio optimization

In [4]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
from utils import *
from portfolio_opt import *
warnings.filterwarnings('ignore')

In [5]:
df = download_stock_data(market_country= 'IT', n_years= 15)
df.head(10)

[*********************100%%**********************]  365 of 365 completed
ERROR:yfinance:
71 Failed downloads:
ERROR:yfinance:['COV.MI', 'FAR.MI', 'LIT.MI', 'SEB.MI', 'MS.MI', 'ACS.MI', 'PSF.MI', 'PLT.MI', 'BLUE.MI', 'DIG.MI', 'AST.MI', 'VTY.MI', 'BDBR.MI', 'AED.MI', 'ISG.MI', 'PRI.MI', 'GCL.MI', 'BEST.MI', 'EXSY.MI', 'CERV.MI', 'ESI.MI', 'SIO.MI', 'TIS.MI', 'LD.MI', 'AGL.MI', 'FKR.MI', 'BCM.MI', 'SOU.MI', 'PAN.MI', 'FNL.MI', 'EVS.MI', 'ATH.MI', 'FHR.MI', 'AMM.MI', 'ASSI.MI', 'REEVO.MI', 'BFE.MI', 'EMC.MI', 'SITI.MI', 'ROS.MI', 'IKG.MI', 'ATL.MI', 'EC.MI', 'WMC.MI', 'AT.MI', 'SICT.MI', 'DEA.MI', 'ISGS.MI', 'TTT.MI', 'SGR.MI', 'NET.MI', 'MOLD.MI', 'YIV.MI', 'PITE.MI', 'LBM.MI', 'KI.MI', 'PSA.MI', 'BOR.MI', 'CRES.MI', 'RM.MI', 'CARR.MI', 'ELIN.MI', 'FED.MI', 'TAS.MI', 'ASR.MI', 'FUL.MI', 'MT.MI', 'CVAL.MI', 'KIP.MI', 'MAIL.MI', 'BIM.MI']: Exception('%ticker%: No timezone found, symbol may be delisted')


adj close    close       high       low     open  \
date       ticker                                                       
2009-01-21 A2A.MI     0.621934   1.3100   1.350000  1.287000   1.2930   
           ACE.MI     5.034316  10.0400  10.250000  9.940000  10.1000   
           AEF.MI     0.632889   0.6400   0.654500  0.628000   0.6505   
           ALW.MI     1.210018   1.5980   1.598000  1.550000   1.5500   
           AMP.MI     0.696417   0.7750   0.790000  0.750000   0.7790   
           ARN.MI     2.933377   3.9400   3.980000  3.835000   3.9100   
           ASC.MI     0.667706   1.5240   1.524000  1.502000   1.5030   
           AUTME.MI   0.995481   8.9500   8.950000  8.950000   8.9500   
           AZM.MI     1.711770   3.3873   3.460538  3.286596   3.3873   
           B.MI       2.265268   2.3000   2.425000  2.117500   2.4250   

                        volume  
date       ticker               
2009-01-21 A2A.MI    6763757.0  
           ACE.MI     234275.0  
           AEF.MI      99541.0  
           ALW.MI       3800.0  
           AMP.MI     235141.0  
           ARN.MI      23182.0  
           ASC.MI     106329.0  
           AUTME.MI        9.0  
           AZM.MI     252372.0  
           B.MI       218741.0

In [6]:
from metrics import StockMeasures

stock_measures = StockMeasures(df)
stock_measures.calculate_measures('all')
stock_measures.measures

garman_klass_vol        rsi    bb_low    bb_mid   bb_high  \
date       ticker                                                               
2009-01-21 A2A.MI          -0.205779        NaN       NaN       NaN       NaN   
           ACE.MI          -0.186794        NaN       NaN       NaN       NaN   
           AEF.MI           0.000563        NaN       NaN       NaN       NaN   
           ALW.MI          -0.023221        NaN       NaN       NaN       NaN   
           AMP.MI          -0.003501        NaN       NaN       NaN       NaN   
...                              ...        ...       ...       ...       ...   
2024-01-17 XPR.MI           0.000051  28.012593  0.970747  0.981876  0.993005   
           YRM.MI           0.000000  64.559293  3.842027  3.904416  3.966805   
           YSFT.MI          0.000000  33.205884  0.586994  0.648075  0.709157   
           ZUC.MI           0.000416  55.985222  1.160494  1.187902  1.215310   
           ZV.MI            0.000403  43.790805  2.669073  2.708984  2.748895   

                         atr      macd  dollar_volume  
date       ticker                                      
2009-01-21 A2A.MI        NaN       NaN       4.206614  
           ACE.MI        NaN       NaN       1.179414  
           AEF.MI        NaN       NaN       0.062998  
           ALW.MI        NaN       NaN       0.004598  
           AMP.MI        NaN       NaN       0.163756  
...                      ...       ...            ...  
2024-01-17 XPR.MI  -1.045554 -1.045554       0.004988  
           YRM.MI   5.112249  5.112249       0.000000  
           YSFT.MI -0.119722 -0.119722       0.000000  
           ZUC.MI  -0.535774 -0.535774       0.011585  
           ZV.MI    1.032335  1.032335       0.506794  

[783744 rows x 8 columns]

Now we add to the feature dataframe the columns of adjusted close and volume.

In [7]:
data_fit = stock_measures.measures.copy()

data_fit["adj close"] = df["adj close"]

data_fit.head()

garman_klass_vol  rsi  bb_low  bb_mid  bb_high  atr  macd  \
date       ticker                                                              
2009-01-21 A2A.MI         -0.205779  NaN     NaN     NaN      NaN  NaN   NaN   
           ACE.MI         -0.186794  NaN     NaN     NaN      NaN  NaN   NaN   
           AEF.MI          0.000563  NaN     NaN     NaN      NaN  NaN   NaN   
           ALW.MI         -0.023221  NaN     NaN     NaN      NaN  NaN   NaN   
           AMP.MI         -0.003501  NaN     NaN     NaN      NaN  NaN   NaN   

                   dollar_volume  adj close  
date       ticker                            
2009-01-21 A2A.MI       4.206614   0.621934  
           ACE.MI       1.179414   5.034316  
           AEF.MI       0.062998   0.632889  
           ALW.MI       0.004598   1.210018  
           AMP.MI       0.163756   0.696417

In [8]:
data = aggregate_data_by_timestep(data_fit, timestep="M")
data

dollar_volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                          
2009-02-28 A2A.MI        5.388844         -0.242996  29.895133  0.438605   
           ACE.MI        4.945657         -0.200020  48.305195  1.716534   
           AEF.MI        0.049393          0.001423  13.780782  0.403250   
           ALW.MI        0.003315         -0.030016  48.238219  0.772448   
           AMP.MI        0.213964         -0.002529  59.323456  0.559967   
...                           ...               ...        ...       ...   
2024-01-31 VNT.MI        0.029490          0.000226  51.895720  1.154522   
           PHIL.MI       0.099626          0.000144  43.322341  2.914633   
           AIW.MI        0.043136          0.001048  45.032086  1.596137   
           CASA.MI       0.004990          0.004304  42.625333  0.534312   
           ABP.MI        0.018958          0.000000  56.308604  1.596578   

                      bb_mid   bb_high       atr      macd  adj close  
date       ticker                                                      
2009-02-28 A2A.MI   0.495893  0.553181  3.949591  3.949591   0.541225  
           ACE.MI   1.782441  1.848348  2.186334  2.186334   4.924003  
           AEF.MI   0.438986  0.474723 -0.897089 -0.897089   0.480600  
           ALW.MI   0.789138  0.805829 -0.479142 -0.479142   1.196388  
           AMP.MI   0.596702  0.633436 -0.949421 -0.949421   0.834801  
...                      ...       ...       ...       ...        ...  
2024-01-31 VNT.MI   1.216511  1.278501 -1.336676 -1.336676   2.330000  
           PHIL.MI  2.946025  2.977417 -0.553152 -0.553152  17.549999  
           AIW.MI   1.622171  1.648205 -1.234740 -1.234740   3.960000  
           CASA.MI  0.557973  0.581634 -0.726349 -0.726349   0.750000  
           ABP.MI   1.606821  1.617064 -1.093023 -1.093023   4.000000  

[37009 rows x 9 columns]

In [9]:
data = cut_low_volume(data, window= 5, min_years= 1, threshold= 150)
data

garman_klass_vol        rsi    bb_low    bb_mid   bb_high  \
date       ticker                                                               
2010-01-31 A2A.MI          -0.183943  45.235827  0.530225  0.545594  0.560963   
           ACE.MI          -0.148387  60.880538  1.576775  1.609739  1.642703   
           AEF.MI           0.002169  41.734402  0.385759  0.411332  0.436906   
           ALW.MI          -0.020303  73.552606  0.841973  1.164751  1.487529   
           AMP.MI           0.001802  65.904353  1.272003  1.322853  1.373703   
...                              ...        ...       ...       ...       ...   
2024-01-31 GVS.MI           0.000364  54.795869  1.815071  1.863784  1.912496   
           CY4.MI           0.000026  38.615187  2.136606  2.188521  2.240437   
           CML.MI           0.000035  49.191899  1.477685  1.504542  1.531400   
           VNT.MI           0.000226  51.895720  1.154522  1.216511  1.278501   
           PHIL.MI          0.000144  43.322341  2.914633  2.946025  2.977417   

                         atr      macd  adj close  
date       ticker                                  
2010-01-31 A2A.MI   0.028157  0.028157   0.702341  
           ACE.MI  -1.079001 -1.079001   4.189543  
           AEF.MI  -1.223799 -1.223799   0.494000  
           ALW.MI   6.181246  6.181246   3.149983  
           AMP.MI  -0.792221 -0.792221   3.028289  
...                      ...       ...        ...  
2024-01-31 GVS.MI  -1.331289 -1.331289   5.530000  
           CY4.MI  -1.235511 -1.235511   7.470000  
           CML.MI  -0.461287 -0.461287   3.440000  
           VNT.MI  -1.336676 -1.336676   2.330000  
           PHIL.MI -0.553152 -0.553152  17.549999  

[24648 rows x 8 columns]

In [10]:
data = aggregate_returns(data, outlier_cutoff= 0.005, lags= [1, 2, 3, 6, 9, 12])
data

garman_klass_vol        rsi    bb_low    bb_mid   bb_high  \
date       ticker                                                               
2011-01-31 A2A.MI          -0.144618  54.824198  0.430308  0.452692  0.475076   
           ACE.MI          -0.148660  43.899351  1.685565  1.706929  1.728294   
           AEF.MI           0.000628  74.812548  0.366369  0.452924  0.539478   
           ALW.MI          -0.025259  48.444151  1.313535  1.333516  1.353497   
           AMP.MI          -0.001798  47.567799  1.448921  1.474768  1.500615   
...                              ...        ...       ...       ...       ...   
2024-01-31 GVS.MI           0.000364  54.795869  1.815071  1.863784  1.912496   
           CY4.MI           0.000026  38.615187  2.136606  2.188521  2.240437   
           CML.MI           0.000035  49.191899  1.477685  1.504542  1.531400   
           VNT.MI           0.000226  51.895720  1.154522  1.216511  1.278501   
           PHIL.MI          0.000144  43.322341  2.914633  2.946025  2.977417   

                         atr      macd  adj close  return_1m  return_2m  \
date       ticker                                                         
2011-01-31 A2A.MI  -0.753402 -0.753402   0.585371   0.054422   0.048471   
           ACE.MI  -1.228021 -1.228021   4.470105  -0.036628  -0.010397   
           AEF.MI  -0.939304 -0.939304   0.659000   0.342159   0.316029   
           ALW.MI   1.018053  1.018053   2.774193   0.017143   0.017292   
           AMP.MI  -0.874371 -0.874371   3.355073  -0.010040   0.005453   
...                      ...       ...        ...        ...        ...   
2024-01-31 GVS.MI  -1.331289 -1.331289   5.530000  -0.008072  -0.007598   
           CY4.MI  -1.235511 -1.235511   7.470000  -0.086797  -0.032482   
           CML.MI  -0.461287 -0.461287   3.440000   0.000000   0.017912   
           VNT.MI  -1.336676 -1.336676   2.330000  -0.068000   0.068717   
           PHIL.MI -0.553152 -0.553152  17.549999  -0.051351  -0.022041   

                    return_3m  return_6m  return_9m  return_12m  
date       ticker                                                
2011-01-31 A2A.MI   -0.025106  -0.009794  -0.011782   -0.015066  
           ACE.MI   -0.011210  -0.010375   0.008388    0.005416  
           AEF.MI    0.151155   0.122178   0.069819    0.024306  
           ALW.MI   -0.037096   0.009308  -0.003904   -0.010531  
           AMP.MI   -0.015084  -0.002790  -0.005889    0.008576  
...                       ...        ...        ...         ...  
2024-01-31 GVS.MI    0.094303  -0.007199  -0.012183    0.011064  
           CY4.MI    0.009550  -0.012998  -0.016668   -0.018445  
           CML.MI    0.005882  -0.007088  -0.007165    0.017606  
           VNT.MI    0.081808  -0.023661  -0.038850   -0.055821  
           PHIL.MI   0.002865   0.015530   0.019500    0.014075  

[21983 rows x 14 columns]

In [11]:
betas = get_famafrench_factors(data)
betas

Mkt-RF       SMB       HML       RMW       CMA
date       ticker                                                   
2011-01-31 A2A.MI        NaN       NaN       NaN       NaN       NaN
           ACE.MI        NaN       NaN       NaN       NaN       NaN
           AEF.MI        NaN       NaN       NaN       NaN       NaN
           ALW.MI        NaN       NaN       NaN       NaN       NaN
           AMP.MI        NaN       NaN       NaN       NaN       NaN
...                      ...       ...       ...       ...       ...
2023-11-30 VNT.MI  -0.750434  1.762789  0.213427 -2.765043 -1.625426
           WBD.MI   1.001884 -0.508569 -0.218046  0.583750  0.076180
           WIIT.MI  1.774518  0.222815 -0.681203 -0.099016  0.891083
           XPR.MI   1.018129  0.037005  0.958183  1.432101 -1.191170
           ZV.MI    0.820966  0.957794  0.274431  0.961689 -0.122612

[21659 rows x 5 columns]

The result will be our feature dataset

In [12]:
ff_factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']

# we merge the previously dataset with the one that we have for the betas factors.

data = (data.join(betas.groupby('ticker').shift()))

data.loc[:, ff_factors] = data.groupby('ticker', group_keys=False)[ff_factors].apply(lambda x: x.fillna(x.mean()))

data = data.drop('adj close', axis=1)

feature_data = data.dropna()

feature_data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 21650 entries, (Timestamp('2011-01-31 00:00:00'), 'A2A.MI') to (Timestamp('2024-01-31 00:00:00'), 'SCF.MI')
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   garman_klass_vol  21650 non-null  float64
 1   rsi               21650 non-null  float64
 2   bb_low            21650 non-null  float64
 3   bb_mid            21650 non-null  float64
 4   bb_high           21650 non-null  float64
 5   atr               21650 non-null  float64
 6   macd              21650 non-null  float64
 7   return_1m         21650 non-null  float64
 8   return_2m         21650 non-null  float64
 9   return_3m         21650 non-null  float64
 10  return_6m         21650 non-null  float64
 11  return_9m         21650 non-null  float64
 12  return_12m        21650 non-null  float64
 13  Mkt-RF            21650 non-null  float64
 14  SMB               21650 non-null  float64
 15  HML  

In [13]:
feature_data

garman_klass_vol        rsi    bb_low    bb_mid   bb_high  \
date       ticker                                                               
2011-01-31 A2A.MI          -0.144618  54.824198  0.430308  0.452692  0.475076   
           ACE.MI          -0.148660  43.899351  1.685565  1.706929  1.728294   
           AEF.MI           0.000628  74.812548  0.366369  0.452924  0.539478   
           ALW.MI          -0.025259  48.444151  1.313535  1.333516  1.353497   
           AMP.MI          -0.001798  47.567799  1.448921  1.474768  1.500615   
...                              ...        ...       ...       ...       ...   
2024-01-31 AV.MI            0.001084  28.097331  0.885751  0.995473  1.105195   
           NEXI.MI          0.000199  45.637794  2.077074  2.111339  2.145603   
           RLT.MI           0.000629  55.362671  1.113773  1.140305  1.166837   
           NWL.MI           0.000572  70.599574  2.072270  2.161766  2.251263   
           SCF.MI           0.000310  48.280073  3.196450  3.224363  3.252276   

                         atr      macd  return_1m  return_2m  return_3m  \
date       ticker                                                         
2011-01-31 A2A.MI  -0.753402 -0.753402   0.054422   0.048471  -0.025106   
           ACE.MI  -1.228021 -1.228021  -0.036628  -0.010397  -0.011210   
           AEF.MI  -0.939304 -0.939304   0.342159   0.316029   0.151155   
           ALW.MI   1.018053  1.018053   0.017143   0.017292  -0.037096   
           AMP.MI  -0.874371 -0.874371  -0.010040   0.005453  -0.015084   
...                      ...       ...        ...        ...        ...   
2024-01-31 AV.MI   -1.661204 -1.661204  -0.265659  -0.292893  -0.199372   
           NEXI.MI -1.262822 -1.262822  -0.056441  -0.011394   0.084667   
           RLT.MI  -0.395725 -0.395725   0.037915   0.009259   0.032437   
           NWL.MI   0.883437  0.883437   0.029262   0.113911   0.110652   
           SCF.MI  -0.157439 -0.157439  -0.038540   0.004246   0.018226   

                    return_6m  return_9m  return_12m    Mkt-RF       SMB  \
date       ticker                                                          
2011-01-31 A2A.MI   -0.009794  -0.011782   -0.015066  0.848265 -0.016891   
           ACE.MI   -0.010375   0.008388    0.005416  0.614947 -0.223752   
           AEF.MI    0.122178   0.069819    0.024306  1.206486 -0.858825   
           ALW.MI    0.009308  -0.003904   -0.010531  0.547734  0.025552   
           AMP.MI   -0.002790  -0.005889    0.008576  0.412608  0.052982   
...                       ...        ...         ...       ...       ...   
2024-01-31 AV.MI    -0.217515  -0.155621   -0.137530  1.375832  0.730531   
           NEXI.MI  -0.019699  -0.007914   -0.011823  0.831170  0.423199   
           RLT.MI    0.000000   0.002050   -0.002802  1.327356 -0.409430   
           NWL.MI    0.046756   0.044868    0.045007  0.384245 -0.641617   
           SCF.MI    0.001059   0.015095    0.025565  1.393243 -1.164708   

                         HML       RMW       CMA  
date       ticker                                 
2011-01-31 A2A.MI   0.001174 -0.110293 -0.133055  
           ACE.MI   0.917002  0.443067 -1.461194  
           AEF.MI   0.050493 -1.434203 -0.297779  
           ALW.MI   0.631947  0.296216 -1.395105  
           AMP.MI  -0.263504 -0.442342 -0.242560  
...                      ...       ...       ...  
2024-01-31 AV.MI    0.959399  1.128100 -0.688896  
           NEXI.MI  0.584257 -0.176961 -0.708741  
           RLT.MI   1.094946 -0.104648 -0.709250  
           NWL.MI   0.400582 -0.058775 -0.625674  
           SCF.MI   0.119953  0.838365 -0.187089  

[21650 rows x 18 columns]

## Clustering methods

We will fit a clustering scheme for each month and get the right stocks that we want.

Totally unsupervised problem.


Best number of clusters = 3. I will choose to take the clusters where the centroid of that is higher in rsi and lower in atr.

In [14]:
best_rsi_data_M = data.copy().dropna().groupby('date', group_keys=False).apply(lambda df: kmeans_best_rsi(df, 3))

We want to create a list of the stocks for the next month.

In [15]:
M_chosen_stocks = list_monthly_stocks(best_rsi_data_M)

stocks = list(np.unique([s for stocks_list in M_chosen_stocks.values() for s in stocks_list]))

# new_df = df[["adj close"]].reset_index(level= 0).loc[stocks]#.reset_index().set_index(["date", "ticker"])
new_df = df[["adj close"]].reset_index(level= 0).loc[stocks].reset_index().pivot(index= "date", columns= "ticker")

new_df.head()

adj close                                                        \
ticker        A2A.MI    ACE.MI ADB.MI    AEF.MI    ALW.MI AMB.MI    AMP.MI   
date                                                                         
2009-01-21  0.621934  5.034316    NaN  0.632889  1.210018    NaN  0.696417   
2009-01-22  0.590600  5.099502    NaN  0.623000  1.211532    NaN  0.721128   
2009-01-23  0.611490  5.114544    NaN  0.613111  1.210775    NaN  0.725172   
2009-01-26  0.631430  5.011751    NaN  0.592839  1.246364    NaN  0.754826   
2009-01-27  0.639975  4.929017    NaN  0.583444  1.241820    NaN  0.806046   

                                        ...                            \
ticker     ANIM.MI    ARN.MI    ASC.MI  ...     US.MI VE.MI    VIA.MI   
date                                    ...                             
2009-01-21     NaN  2.933377  0.667706  ...  8.845475   NaN  1.045483   
2009-01-22     NaN  2.963158  0.665954  ...  7.981224   NaN       NaN   
2009-01-23     NaN  2.955712  0.656753  ...  7.640761   NaN  1.054718   
2009-01-26     NaN  2.955712  0.664639  ...  7.856825   NaN  1.048782   
2009-01-27     NaN  2.903597  0.664201  ...  7.810992   NaN       NaN   

                                                                       \
ticker        VLS.MI    WBD.MI   WBDR.MI WIIT.MI    XPR.MI     ZUC.MI   
date                                                                    
2009-01-21  1.476517  0.607429  3.625874     NaN  0.652050  14.285087   
2009-01-22  1.495609  0.597952  3.623085     NaN  0.640885  13.402740   
2009-01-23  1.489245  0.581138       NaN     NaN  0.624509  13.179361   
2009-01-26  1.489245  0.600703       NaN     NaN  0.652795  13.168192   
2009-01-27  1.489245  0.602843       NaN     NaN  0.649817  13.179361   

                      
ticker         ZV.MI  
date                  
2009-01-21  1.662834  
2009-01-22  1.689997  
2009-01-23  1.634490  
2009-01-26  1.662834  
2009-01-27  1.681730  

[5 rows x 194 columns]

# QUI

In [31]:
def buy_or_sell_(dataframe, start_date, port_weights, choosen_stocks, off_set_days = 100):
        closing_prices = dataframe["adj close"]
        start_date_timestamp = pd.to_datetime(start_date)
        end_date = start_date_timestamp-pd.offsets.Day(260*5)
        past_data = closing_prices.loc[end_date: start_date_timestamp, choosen_stocks]

        angular_coeff_df = pd.DataFrame(index=choosen_stocks, columns=['Angular_Coefficient'])
        for stock in choosen_stocks:
            X_stock = past_data[stock].values.reshape(-1,1)

            scaler = MinMaxScaler()
            X_scaled = scaler.fit_transform(X_stock).reshape(-1)
            # df_scaled = pd.DataFrame(df_scaled, index= df.index, columns= df.columns.get_level_values(level= 1))
            N = len(X_scaled)
            X = np.zeros((N- off_set_days, off_set_days))
            y = np.zeros((N- off_set_days,))
            for i in range(N- off_set_days):
              X[i, :] = X_scaled[i : i + off_set_days]   # copy 0-99
              y[i] = X_scaled[i + off_set_days]       # copy 100

            ESNmodel = Sequential([
                                  layers.Input((off_set_days, 1)),
                                  tfa.layers.ESN(units= 50,
                                                connectivity= 0.2,
                                                return_sequences= True),
                                  tfa.layers.ESN(units= 50,
                                                connectivity= 0.2,
                                                return_sequences= False),
                                  layers.Dense(25),
                                  layers.Dense(1)
                              ],
                                  name= f"esn_{stock}_{start_date}"
            )
            ESNmodel.compile(optimizer = "adam", loss= "mean_squared_error")
            ESNmodel.fit(X, y, batch_size = 5, epochs= 3, verbose= 0) ##

            test_start = dt.datetime.strptime(start_date, "%Y-%m-%d")
            test_end = test_start + dt.timedelta(30)

            pred = np.append(
                X[-1],
                np.zeros((30,))
            )

            for i in range(30):
              pred[i+100] = ESNmodel.predict(pred[i:i+100].reshape(1,-1), verbose= 0)

            first_point = pred[100]
            last_point = pred[-1]

            slope = (last_point - first_point) / 30

            angular_coeff_df.loc[stock, 'Angular_Coefficient'] = slope

        returns = []
        for i in range(len(angular_coeff_df)):
            returns.append(angular_coeff_df.iloc[i,0]*port_weights.T.iloc[i,0])
        return np.sum(returns)


In [41]:
def daily_portfolio_return_(choosen_stocks: dict, fresh_data: pd.DataFrame, stop_loss = False, off_set_days = 10):
    # Fresh data = c'è la data, e le colonne solo per le choosen stocks
    counter = 0
    # we calculate the returns of the model
    returns_dataframe = np.log(fresh_data['adj close']).diff()

    portfolio_df = pd.DataFrame()
     # we loop over the dates in our portfolio. Staring date of each month
    for start_date in tqdm(choosen_stocks.keys()):

        # try:

            # end of date of each month
            end_date = (pd.to_datetime(start_date)+pd.offsets.MonthEnd(0)).strftime('%Y-%m-%d')

            # we select the stocks for the month.
            cols = choosen_stocks[start_date]

            # now we calculate the weights for the optimization. We need a year of previous data:

            #starting date for the optimization
            optimization_start_date = (pd.to_datetime(start_date)-pd.DateOffset(months=12)).strftime('%Y-%m-%d')

            # last date of the optimization
            optimization_end_date = (pd.to_datetime(start_date)-pd.DateOffset(days=1)).strftime('%Y-%m-%d')

            # select all the data inside the window of one jear for optimizing the portfolio
            optimization_df = fresh_data[optimization_start_date:optimization_end_date]['adj close'][cols]

            success = False
            try:
                # the lower bound is half of the weight of an equally weighted portfolio.
                # Lower bound for the weights is now  around 0.01
                weights = optimize_weights(prices=optimization_df,
                                    lower_bound=round(1/(len(optimization_df.columns)*2),3))

                # we store the weights as a pd dataframe to multiply the weights for each stock by it's return
                # for every day

                weights = pd.DataFrame(weights, index=pd.Series(0))

                success = True
            except:
                print(f'Max Sharpe Optimization failed for {start_date}, Continuing with Equal-Weights')

            if success==False:
                weights = pd.DataFrame([1/len(optimization_df.columns) for i in range(len(optimization_df.columns))],
                                        index=optimization_df.columns.tolist(),
                                        columns=pd.Series(0)).T
            if stop_loss:
                slope_returns = buy_or_sell_(fresh_data,start_date=start_date,
                                            port_weights=weights,
                                            choosen_stocks=cols,
                                            off_set_days = off_set_days)

        # select the stocks for a month
        # temp_df = returns_dataframe[start_date:end_date]
            temp_df = returns_dataframe[start_date:end_date]

            # create a new dataframe where u have or that time period the daily return of that stock and the monthly weights.
            # only for the selected stocks of that month.
            temp_df = temp_df.stack().to_frame('return').reset_index(level=0)\
                    .merge(weights.stack().to_frame('weight').reset_index(level=0, drop=True),
                            left_index=True,
                            right_index=True)
            temp_df.reset_index().set_index(['date', 'ticker']).unstack().stack()
            # display(temp_df)


            # temp_df.index.names = ['date', 'ticker']

            # we multiply the return for each day with the montly weights for each stock.
            temp_df['weighted_return'] = temp_df['return']*temp_df['weight']

            # we sum the daily weighted returns of our portfolio and then we store it as a strategy return of each day.
            temp_df = temp_df.groupby(level=0)['weighted_return'].sum().to_frame('Strategy Return')

            if stop_loss and slope_returns < 0. :
                temp_df["Strategy Return"] = 0.
                counter +=1
            # we obtain the dayly strategy return of our portfolio.
            portfolio_df = pd.concat([portfolio_df, temp_df], axis=0)


        # except Exception as e:
        #     print(e)
    print(counter)
    portfolio_df
    return portfolio_df

In [43]:
from sklearn.preprocessing import MinMaxScaler
from keras import Sequential, layers
import tensorflow_addons as tfa
from tqdm import tqdm

In [44]:
portftolio_df_stop_loss = daily_portfolio_return_(M_chosen_stocks, new_df, off_set_days = 100, stop_loss = True)

  1%|▏         | 2/157 [11:05<14:19:33, 332.73s/it]


KeyboardInterrupt: 

In [39]:
portftolio_df_stop_loss

NameError: name 'portftolio_df_stop_loss' is not defined

In [ ]:
df_to_opt = df.reset_index(level= 0).loc[stocks_to_optimize][["date", "adj close"]].reset_index().set_index(["date", "ticker"]).unstack(level= 1)
df_to_opt.head()


In [ ]:
df_to_opt.iloc[:,:5].plot()

In [ ]:
import tensorflow_addons as tfa
import keras
from keras import layers
from keras import Sequential

in_sequence = 100
hidden = 50


ESNmodel = Sequential([
    layers.Input((in_sequence, 1)),
    tfa.layers.ESN(units= hidden,
                   connectivity= 0.2,
                   return_sequences= True),
    tfa.layers.ESN(units= hidden,
                   connectivity= 0.2,
                   return_sequences= False),
    layers.Dense(hidden // 2),
    layers.Dense(1)
],
                      name= "Echo_State_Network_model")

LSTMmodel = Sequential([
    layers.Input((in_sequence, 1)),
    layers.LSTM(units= hidden,
                return_sequences= True),
    layers.LSTM(units= hidden,
                return_sequences= False),
    layers.Dense(hidden // 2),
    layers.Dense(1)
],
                       name= "LSTM_model")

ESNmodel.compile(optimizer ="adam",loss="mean_squared_error")
print(ESNmodel.summary())
LSTMmodel.compile(optimizer ="adam",loss="mean_squared_error")
print(LSTMmodel.summary())



In [ ]:
from sklearn.preprocessing import MinMaxScaler
def minmax_data(df):
  # df_scaled = df.copy(deep= True)
  scalers = {}
  # for ticker in df.columns.get_level_values(level= 1):
  # for ticker in df.columns:
  #     scaler = MinMaxScaler()
  #     df_scaled.loc[:, ticker] = scaler.fit_transform(df.loc[:,ticker].values.reshape(1, -1))
  #     scalers[ticker] = scaler
  scaler = MinMaxScaler()
  df_scaled = scaler.fit_transform(df)
  df_scaled = pd.DataFrame(df_scaled, index= df.index, columns= df.columns.get_level_values(level= 1))

  return scaler, df_scaled

scaler, df_scaled = minmax_data(df_to_opt)
# df_scaled.iloc[:, :5].plot()
# plt.show()
df_scaled[["AZM.MI"]].head(100)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
seq = df_scaled.loc[:,"AZM.MI"].values
N = len(seq)
X = np.zeros((N-100, 100))
y = np.zeros((N-100,))
for i in range(N-100):
  X[i,:] = seq[i:i+100]   # copy 0-99
  y[i] = seq[i+100]       # copy 100

assert((X[1:,-1] == y[:-1]).all()) # the last element of the previous sequence is actually the ground truth

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size= 0.8, shuffle= False)

In [ ]:
ESNmodel.fit(X_train, y_train, batch_size = 5, epochs= 2);
LSTMmodel.fit(X_train, y_train, batch_size = 5, epochs= 2);

In [ ]:
y_pred_esn_ = ESNmodel.predict(X_test)
y_pred_lstm_ = LSTMmodel.predict(X_test)

In [ ]:
scaler.min_

In [ ]:
def reverse_scaling(y, scaler, name):
  idx = np.argwhere(df_scaled.columns.values == name).item()

  min_val = scaler.data_min_[idx]
  scale = 1 / (scaler.data_max_[idx] - scaler.data_min_[idx])
  return (y - scaler.min_[idx]) / scaler.scale_[idx]
  # inverse_scaled_data[:, feature_index] = inverse_scaled_data[:, feature_index] * (max_val - min_val) + min_val

y_pred_esn = reverse_scaling(y_pred_esn_, scaler, "AZM.MI")
y_pred_lstm = reverse_scaling(y_pred_lstm_, scaler, "AZM.MI")
y_pred_gt = reverse_scaling(y_test, scaler, "AZM.MI")

In [ ]:
train = df_to_opt[[('adj close',   'AZM.MI')]].iloc[:100+len(y_train)]
valid = df_to_opt[[('adj close',   'AZM.MI')]].iloc[100+len(y_train):]

valid["ESN"] = y_pred_esn
valid["LSTM"] = y_pred_lstm

In [ ]:
MSE_esn = np.mean((y_pred_esn - y_pred_gt)**2)
MSE_lstm = np.mean((y_pred_lstm - y_pred_gt)**2)

print(f"Mean squared error ESN  = {MSE_esn}")
print(f"Mean squared error LSTM = {MSE_lstm}")

In [ ]:
(y_pred_lstm - y_test)

In [ ]:
valid.plot()

In [ ]:
train.plot()

In [ ]:
import tensorflow as tf

In [ ]:
prediction_days = 30

test_start = dt.datetime(2023, 1, 1)
test_end = test_start + dt.timedelta(prediction_days)

pred = np.append(
    df_scaled.loc[: test_start, "AZM.MI"].iloc[-100:],
    np.zeros((prediction_days,))
)

model = LSTMmodel

for i in range(prediction_days):
  pred[i+100] = model.predict(pred[i:i+100].reshape(1,-1))

In [ ]:
comparison = pd.DataFrame()
comparison["true data"] = df_scaled.loc[test_start:, "AZM.MI"].iloc[:prediction_days]
comparison["predictions"] = pred[-prediction_days:]
comparison.plot()

## Prova optimization

In [ ]:
def prepare_train_fit(dataframe, start_date, port_weights, choosen_stocks, off_set_days = 30):
  end_train = pd.to_datetime(start_date)
  start_train = start_date - pd.Offset.year(5)

  past_data = download_stock_data('Italy', )
